In [ ]:
day_obs = "2025-04-18"
time_gap = 40

# SQMiner Logs for LSSTCam (consdb) visit gaps {{ params.day_obs }} (gap > {{ params.time_gap }})

In [ ]:
%%capture
pip install --user git+https://github.com/lsst-sims/rubin_nights --no-deps --quiet

In [ ]:
import numpy as np
import pandas as pd
from astropy.time import Time, TimeDelta
import astropy.units as u
from zoneinfo import ZoneInfo
from IPython.display import display, HTML
from rubin_scheduler.utils import Site
from astroplan import Observer

from rubin_nights import scriptqueue
from rubin_nights import scriptqueue_formatting
from rubin_nights import connections

tz_utc = ZoneInfo("UTC")

In [ ]:
if day_obs.lower() == "today":
    # Shift the 12hour offset following the definition of day_obs in https://sitcomtn-032.lsst.io/    
    # Drop the hours, minutes, seconds to get the ISO formatted day_obs
    day_obs = Time(np.floor(Time.now().mjd - 0.5), format='mjd', scale='utc').iso[0:10]
 
elif day_obs.lower() == "yesterday":
    # Shift the 12hour offset following the definition of day_obs in https://sitcomtn-032.lsst.io/
    # Drop the hours, minutes, seconds to get the ISO fromatted day_obs
    day_obs = (Time(np.floor(Time.now().mjd - 0.5), format='mjd', scale='utc') - TimeDelta(1, format='jd')).iso[0:10]

else:
    # test that day_obs is a proper day_obs
    try:
        test_day_obs = Time(f"{day_obs}T12:00:00", format='isot', scale='utc')
    except ValueError:
        msg = "day_obs should be a date formatted as YYYY-MM-DD"
        raise ValueError(msg)

In [ ]:
endpoints = connections.get_clients()

observer = Observer(location=Site('LSST').to_earth_location())
day_obs_time = Time(f"{day_obs}T12:00:00", format='isot', scale='tai')
sunset = Time(observer.sun_set_time(day_obs_time, which='next', horizon=-12*u.deg), format='jd')
sunrise = Time(observer.sun_rise_time(day_obs_time, which='next', horizon=-12*u.deg), format='jd')
print(day_obs, 'sunset', sunset.iso,  'sunrise', sunrise.iso)

visits = endpoints['consdb'].get_visits('lsstcam', day_obs, day_obs)
print(f"Found {len(visits)} visits from the ConsDB")
efd_and_messages, log_cols = scriptqueue.get_consolidated_messages(Time(sunset, format='iso'), sunrise)

In [ ]:
display(HTML("<em>Show visits before large gap, the visit with the large gap, and then the next visit after that which is not a BIAS frame</em>"))
print("")

cols = ['exposure_name', 'day_obs', 'seq_num', 'obs_start', 'band', 's_ra', 's_dec', 'sky_rotation', 'visit_gap', 'img_type', 'target_name', 'observation_reason', 'science_program']
vv = visits.query('obs_start_mjd >= @sunset.mjd').iloc[1:]
long_gaps = np.where(vv.visit_gap > time_gap*60)[0]
# Remove first bias after long gap
long_gaps = long_gaps
for i, gap in enumerate(long_gaps):
    gap_interval = vv.iloc[gap].visit_gap / 60
    night_hour = (vv.iloc[gap].obs_start_mjd - sunset.mjd) * 24
    display(HTML(f"<p> <em> Gap {i} ({gap_interval :.1f} minutes) in the {night_hour :.1f} hour after sunset </em>"))
    visit_context = vv.iloc[gap-1:gap+1]
    visit_context = pd.concat([visit_context, visits.query('seq_num > @visit_context.seq_num.max() and img_type != "BIAS"')[0:1]])
    display(visit_context[cols])

In [ ]:
gap_sum = 0
for i, gap in enumerate(long_gaps):
    v1 = vv.iloc[gap-1]
    v2 = vv.iloc[gap]
    t_start = (Time(v1.obs_start, scale='tai') - TimeDelta(10, format='sec')).utc.to_datetime(timezone=tz_utc)
    t_end = (Time(v2.obs_end, scale='tai') + TimeDelta(5, format='sec')).utc.to_datetime(timezone=tz_utc)
    html = scriptqueue_formatting.format_html(efd_and_messages.query('time >= @t_start and time <= @t_end'), log_cols, time_order='oldest first')
    print(" \n ")
    gap_interval = vv.iloc[gap].visit_gap / 60
    gap_sum += gap_interval
    display(HTML(f"<strong> Gap {i} ({gap_interval :.1f} minutes) </strong>"))
    print(f"From visit {v1.exposure_name} at {v1.obs_start} to {v2.exposure_name} at {v2.obs_end}")
    print(" ")
    display(HTML(html))
display(HTML(f"<strong> Sum of gap time {gap_sum/60 :.3f}</strong>"))